In [1]:
import math
from scipy.optimize import minimize
import numpy as np

In [2]:
### tools：

def minfunction(f, x0, bounds):
    """
    求一个函数在边界内的最小值
    f：函数
    x0：初始值
    bounds：边界条件。[(min, max),...]
    """
    res = minimize(f, x0, method='SLSQP', bounds=bounds)
    return res

def maxfunction(f, x0, bounds):
    """
    求一个函数在边界内的最大值
    f：函数
    x0：初始值
    bounds：边界条件。[(min, max),...]
    """
    fun = lambda X : -f(X)
    res = minimize(fun, x0, method='SLSQP', bounds=bounds)
    return res

In [3]:
### 目标函数：

bounds = [(0,400),(0,1000),(340,420),(340,420),(-1000,1000),(0,0.5236)]
X_optimal = [201.784467214523659, 99.9999999999999005,383.071034852773266, 420, -10.9076584514292652, 0.0731482312084287128]
y_optimal =  8853.534016435708
def G17(X):
    y1 = 0
    y2 = 0
    if X[0]>=0 and X[0]<300:
        y1 = 30*X[0]
    else:
        y1 = 31*X[0]
    if X[1]>=0 and X[1]<100:
        y2 = 28*X[1]
    elif X[1]>=100 and X[1]<200:
        y2 = 29*X[1]
    else:
        y2 = 30*X[1]
    y = y1 + y2
    h1 = -X[0]+300-X[2]*X[3]*np.cos(1.48477-X[5])/131.078+0.90798*X[2]*X[2]*np.cos(1.47588)/131.078-0.0001
    h11 = -(-X[0]+300-X[2]*X[3]*np.cos(1.48477-X[5])/131.078+0.90798*X[2]*X[2]*np.cos(1.47588)/131.078)-0.0001
    h2 = -X[1]-X[2]*X[3]*np.cos(1.48477+X[5])/131.078+0.90798*X[3]*X[3]*np.cos(1.47588)/131.078-0.0001
    h22 = -(-X[1]-X[2]*X[3]*np.cos(1.48477+X[5])/131.078+0.90798*X[3]*X[3]*np.cos(1.47588)/131.078)-0.0001
    h3 = -X[4]-X[2]*X[3]*np.sin(1.48477+X[5])/131.078+0.90798*X[3]*X[3]*np.sin(1.47588)/131.078-0.0001
    h33 = -(-X[4]-X[2]*X[3]*np.sin(1.48477+X[5])/131.078+0.90798*X[3]*X[3]*np.sin(1.47588)/131.078)-0.0001
    h4 = 200-X[2]*X[3]*np.sin(1.48477-X[5])/131.078+0.90798*X[2]*X[2]*np.sin(1.47588)/131.078-0.0001
    h44 = -(200-X[2]*X[3]*np.sin(1.48477-X[5])/131.078+0.90798*X[2]*X[2]*np.sin(1.47588)/131.078)-0.0001
        
    return y,h1,h11,h2,h22,h3,h33,h4,h44

In [6]:
solution = [2.920682461476509957e+02,0.000000000000000000e+00,3.882334809580793262e+02,4.200000000000000000e+02,-2.295086174656931988e+01,0.000000000000000000e+00]

In [7]:
G17(solution)

(8762.04738442953,
 -1.6721060571312646e-13,
 -0.0001999999998327894,
 8.92374331287586,
 -8.92394331287586,
 -5.055361773665937e-05,
 -0.00014944638226334063,
 -2.5102058561106422e-14,
 -0.00019999999997489794)

In [4]:
# y = lambda X : G17(X)[0]
# h1 = lambda X : G17(X)[1]
# h2 = lambda X : G17(X)[2]
# h3 = lambda X : G17(X)[3]
# h4 = lambda X : G17(X)[4]
# x0 = [200,200,400,400,0,0]

In [5]:
# maxfunction(h4,x0,bounds)

In [6]:
# 322.20266397217756+754.7251362740523+686.0644874006535+1017.7123118524003

In [7]:
### 适应度函数
def fitnessG17_num(X):
    """
    problem1关于数量惩罚的适应度函数
    X: 对应的解
    epsion：惩罚系数大于fmax-fmin的倍数
    """
    fmax = 42399.99999999925
    fmin = 0
    epsion = 1
    count = np.sum(np.array(G17(X))[1:5]>0)
    
    return -(G17(X)[0]+epsion*(fmax-fmin)*count)  

def fitnessG17_vio(X):
    """
    problem1关于数量惩罚的适应度函数
    X: 对应的解
    epsion：惩罚系数大于fmax-fmin的倍数
    """
    fmax = 42399.99999999925
    fmin = 0.0
    vmax = 2780.704599499283
    e = 0.1
    n = 8
    
    res = G17(X)
    fitness = -res[0]
    count = np.sum(np.array(res)[1:9]>0)
    
    
    if count > 0:
        for i in range(1,len(res)):
            if res[i] > 0:
                cathy = (fmax - fmin)*(e*vmax*n/count+res[i])/(e*vmax)
                fitness = fitness - cathy
    return fitness   

In [8]:
### PSO
class PSO_fit():
    def __init__(self, pN, dim, max_iter, bounds):
        self.wmax = 0.8 #最大的惯性权重
        self.wmin = 0.4 #最小的惯性权重
        self.c1 = 2  #加速因子
        self.c2 = 2  #加速因子
        self.pN = pN  # 粒子数量
        self.dim = dim  # 搜索维度
        self.max_iter = max_iter  # 迭代次数
        self.bounds = bounds
        
        self.X = np.zeros((self.pN, self.dim))  # 所有粒子的位置：一个pN*dim的矩阵
        self.V = np.zeros((self.pN, self.dim))  # 左右粒子的速度：一个pN*dim的矩阵
        self.pbest = np.zeros((self.pN, self.dim))  # 个体经历的最佳位置：一个pN*dim的矩阵
        self.gbest = np.zeros((self.pN, self.dim))      # 全局最佳位置：一个pN*dim的矩阵
        self.p_fit = np.full(self.pN,-10.0e+28)  # 每个个体的历史最佳适应值
        self.g_fit = np.full(self.pN,-10.0e+28)    # 全局最佳适应值
    
    def init_Population(self):
        """
        初始化种群
        :return:
        """
        
        for i in range(self.pN):
            for j in range(self.dim):
                self.X[i][j] = self.bounds[j][0] + (self.bounds[j][1] - self.bounds[j][0]) * np.random.random()
                self.V[i][j] = self.bounds[j][0] + (self.bounds[j][1] - self.bounds[j][0]) * np.random.random()
        for i in range(self.pN):
            tmp = fitnessG17_vio(self.X[i])
            if (tmp > self.p_fit[i]):
                self.p_fit[i] = tmp
                self.pbest[i] = self.X[i]
                
        for i in range(self.pN):
            indexes = np.random.randint(low=0,high=self.pN,size=int(self.pN/100))
            for j in indexes:
                if self.p_fit[j] > self.g_fit[i]:
                    self.g_fit[i] = self.p_fit[j]
                    self.gbest[i] = self.pbest[j]

#             self.pbest[i] = self.X[i]
#             tmp = fitnessG01_num(self.X[i])
#             self.p_fit[i] = tmp
#             if (tmp > self.fit):
#                 self.fit = tmp
#                 self.gbest = self.X[i]
        
    def iterator(self):
        """
        迭代
        """
        for t in range(self.max_iter):
            w = self.wmin + (self.wmax - self.wmin) * np.exp(- (4 * t / self.max_iter) * (4 * t / self.max_iter))
            
            for i in range(self.pN):
                self.V[i] = w*self.V[i]+self.c1*np.random.random()*(self.pbest[i]-self.X[i])+self.c2*np.random.random()*(self.gbest[i]-self.X[i])
                self.X[i] = self.X[i] + self.V[i]
                
                """
                控制在搜索范围内
                """ 
                for j in range(self.dim):
                    if self.X[i][j] < self.bounds[j][0]:
                        self.X[i][j] = self.bounds[j][0]
                    if self.X[i][j] > self.bounds[j][1]:
                        self.X[i][j] = self.bounds[j][1]     
            for i in range(self.pN):
                tmp = fitnessG17_vio(self.X[i])
                if (tmp > self.p_fit[i]):
                    self.p_fit[i] = tmp
                    self.pbest[i] = self.X[i]
            for i in range(self.pN):
                indexes = np.random.randint(low=0,high=self.pN,size=int(self.pN/100))
                for j in indexes:
                    if self.p_fit[j] > self.g_fit[i]:
                        self.g_fit[i] = self.p_fit[j]
                        self.gbest[i] = self.pbest[j]
            print(self.g_fit)  
        return self.gbest, self.g_fit

In [9]:
PSO = PSO_fit(pN=4000, dim=6, max_iter=1000, bounds=bounds)

In [10]:
PSO.init_Population()

In [11]:
gbest, gfit = PSO.iterator()

[-371055.98281351 -406616.46576193 -411598.24853786 ... -405136.44764032
 -385840.12835836 -409146.83093445]
[-371055.98281351 -380465.16314048 -376950.40833953 ... -382584.14438415
 -385840.12835836 -375453.61370785]
[-371055.98281351 -360098.84525037 -376950.40833953 ... -377942.34402879
 -377933.63823997 -370577.77919865]
[-369753.46927272 -360098.84525037 -367583.48447181 ... -362954.39873913
 -367840.51280368 -370577.77919865]
[-364871.11785908 -360098.84525037 -363311.68938831 ... -362954.39873913
 -366645.42359114 -367875.75448862]
[-364871.11785908 -360098.84525037 -363311.68938831 ... -360808.02831705
 -366645.42359114 -363135.96600319]
[-363429.40487285 -357942.920681   -359127.09361898 ... -360808.02831705
 -365217.24373288 -360561.09335598]
[-363429.40487285 -357942.920681   -359127.09361898 ... -358609.74214888
 -363160.58770147 -360561.09335598]
[-362633.17094282 -355559.65604186 -359127.09361898 ... -357765.14792567
 -358518.45936961 -350165.23346029]
[-355416.7876502  -

[-348143.01586536 -348122.20648352 -348115.88986278 ... -348115.88986278
 -348132.82832616 -348140.87841438]
[-348143.01586536 -348122.20648352 -348115.88986278 ... -348115.88986278
 -348132.82832616 -348140.87841438]
[-348143.01586536 -348112.39323052 -348115.88986278 ... -348084.50411255
 -348100.05103463 -348140.87841438]
[-348143.01586536 -348112.39323052 -348115.88986278 ... -348084.50411255
 -348100.05103463 -348130.8474501 ]
[-348124.36312989 -348112.39323052 -348115.88986278 ... -348084.50411255
 -348100.05103463 -348130.8474501 ]
[-348124.36312989 -348112.39323052 -348112.92597359 ... -348084.50411255
 -348100.05103463 -348122.20648352]
[-348124.36312989 -348112.39323052 -348101.52422776 ... -348084.50411255
 -348100.05103463 -348113.83781447]
[-348098.96303679 -348107.7512452  -348101.52422776 ... -348084.50411255
 -348100.05103463 -348113.83781447]
[-348098.96303679 -348107.7512452  -348101.52422776 ... -348084.50411255
 -348100.05103463 -348113.83781447]
[-348098.96303679 -

[-348054.51749583 -348054.72901844 -348054.5934873  ... -348055.12524056
 -348054.4974153  -348054.31774106]
[-348054.51749583 -348054.16082862 -348054.51749583 ... -348054.43332899
 -348054.4974153  -348054.19589203]
[-348054.51749583 -348054.16082862 -348054.51749583 ... -348054.16082862
 -348054.4974153  -348054.19589203]
[-348054.51749583 -348054.0411947  -348054.51749583 ... -348054.16082862
 -348054.17149936 -348053.93856059]
[-348054.51749583 -348054.0411947  -348054.51749583 ... -348054.16082862
 -348054.17149936 -348053.93856059]
[-348053.99813711 -348054.0411947  -348054.51749583 ... -348054.16082862
 -348054.17149936 -348053.93856059]
[-348053.99813711 -348054.0411947  -348054.51749583 ... -348054.16082862
 -348054.17149936 -348053.93856059]
[-348053.81006549 -348054.0411947  -348054.16098812 ... -348054.16082862
 -348054.17149936 -348053.93856059]
[-348053.81006549 -348054.0411947  -348054.16098812 ... -348054.16082862
 -348054.17149936 -348053.93856059]
[-348053.81006549 -

[-8853.53413553 -8853.53408441 -8853.53430831 ... -8853.53426415
 -8853.53420852 -8853.53419483]
[-8853.53413553 -8853.53408441 -8853.53411046 ... -8853.53424623
 -8853.53420852 -8853.53411454]
[-8853.53405795 -8853.53408441 -8853.53411046 ... -8853.53424623
 -8853.53420852 -8853.53411454]
[-8853.53405795 -8853.53408441 -8853.53411046 ... -8853.53424623
 -8853.53420852 -8853.53411454]
[-8853.53405795 -8853.53403776 -8853.53411046 ... -8853.53407247
 -8853.53413719 -8853.53411171]
[-8853.53405795 -8853.53403776 -8853.53411046 ... -8853.53407247
 -8853.53413719 -8853.53411171]
[-8853.53403799 -8853.53403776 -8853.53411046 ... -8853.53407247
 -8853.53406332 -8853.53409557]
[-8853.53403353 -8853.53397943 -8853.53407202 ... -8853.53403182
 -8853.53406332 -8853.53396094]
[-8853.53403353 -8853.53397943 -8853.53398904 ... -8853.53403182
 -8853.5340176  -8853.53396094]
[-8853.53398813 -8853.53397177 -8853.53398904 ... -8853.53403182
 -8853.53401349 -8853.53396094]
[-8853.53397548 -8853.53397138

[-8853.53387483 -8853.53387482 -8853.53387483 ... -8853.53387483
 -8853.53387482 -8853.53387482]
[-8853.53387483 -8853.53387482 -8853.53387483 ... -8853.53387483
 -8853.53387482 -8853.53387482]
[-8853.53387483 -8853.53387482 -8853.53387482 ... -8853.53387481
 -8853.53387482 -8853.53387482]
[-8853.53387482 -8853.53387482 -8853.53387482 ... -8853.53387481
 -8853.53387482 -8853.53387482]
[-8853.53387482 -8853.53387482 -8853.53387482 ... -8853.53387481
 -8853.53387482 -8853.53387482]
[-8853.53387481 -8853.53387482 -8853.53387482 ... -8853.53387481
 -8853.53387482 -8853.53387482]
[-8853.53387481 -8853.53387482 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387482]
[-8853.53387481 -8853.53387482 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387482]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387482]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387482]
[-8853.53387481 -8853.53387481

...

[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481

[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481

[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481

[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]
[-8853.53387481 -8853.53387481

[-8853.53387481 -8853.53387481 -8853.53387481 ... -8853.53387481
 -8853.53387481 -8853.53387481]


In [12]:
PSO.gbest[0]

array([ 2.01784462e+02,  1.00000000e+02,  3.83071035e+02,  4.20000000e+02,
       -1.09076842e+01,  7.31482312e-02])

In [13]:
G17(PSO.gbest[0])

(8853.533874806484,
 -1.0891203845904418e-14,
 -0.0001999999999891088,
 -1.0891203845904418e-14,
 -0.0001999999999891088,
 -0.00014954049632033274,
 -5.0459503679667255e-05,
 -1.3878889628272245e-13,
 -0.0001999999998612111)